In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install -q streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 103.1 MB/s eta 0:00:00


In [3]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input as preprocess_densenet
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_efficientnet
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.vgg19 import preprocess_input as preprocess_vgg19
from PIL import Image
import numpy as np

st.set_page_config(
    page_title="Cataract Detection Ensemble",
    page_icon="👁",
    layout="centered"
)

# Your actual Google Drive model paths:
model_info = {
    'densenet121': {'path': '/content/drive/MyDrive/PROJECT/PROJECT-1/trained models/H5 files/densenet121(99_03).h5',       'preprocess': preprocess_densenet},
    'efficientnetb3': {'path': '/content/drive/MyDrive/PROJECT/PROJECT-1/trained models/H5 files/efficientnetb3(99_84).h5', 'preprocess': preprocess_efficientnet},
    'resnet': {'path': '/content/drive/MyDrive/PROJECT/PROJECT-1/trained models/H5 files/resnet(99_91).h5',                 'preprocess': preprocess_resnet},
    'vgg19': {'path': '/content/drive/MyDrive/PROJECT/PROJECT-1/trained models/H5 files/vgg19(99_89).h5',                   'preprocess': preprocess_vgg19},
}
CLASS_LABELS = ["Immature Cataract", "Mature Cataract", "Normal"]
IMG_SIZE = (224, 224)

@st.cache_resource
def load_models():
    models = {}
    for model_name, item in model_info.items():
        models[model_name] = tf.keras.models.load_model(item['path'], compile=False)
    return models

def preprocess_img(img, pre_func):
    img = img.resize(IMG_SIZE)
    arr = image.img_to_array(img)
    arr = np.expand_dims(arr, axis=0)
    arr = pre_func(arr)
    return arr

def ensemble_predict(image_pil, models):
    all_probs = []
    for model_name, model in models.items():
        pre_func = model_info[model_name]['preprocess']
        processed = preprocess_img(image_pil, pre_func)
        pred = model.predict(processed, verbose=0)[0]
        all_probs.append(pred)
    all_probs = np.array(all_probs)
    avg_probs = all_probs.mean(axis=0)
    best_idx = np.argmax(avg_probs)
    return CLASS_LABELS[best_idx], avg_probs[best_idx]*100, avg_probs

st.title("👁 Cataract Detection System — Ensemble")
st.markdown("Upload an eye image to detect and classify cataracts using 4 models (average confidence shown).")

with st.sidebar:
    st.info("""
        This system uses DenseNet121, EfficientNetB3, ResNet, and VGG19.
        The predicted class is based on **average confidence** across all four models.
        Make sure your models are accessible at the paths above!
    """)

uploaded_file = st.file_uploader(
    "Choose an eye image...", type=['jpg', 'jpeg', 'png']
)
if uploaded_file is not None:
    image_pil = Image.open(uploaded_file).convert('RGB')
    st.image(image_pil, caption='Uploaded Image', use_container_width=True)
    if st.button(' Analyze Image'):
        with st.spinner("Analyzing with all models..."):
            models = load_models()
            result, confidence, avg_probs = ensemble_predict(image_pil, models)
        st.markdown(f"### Prediction: <span style='color:green'>{result}</span>", unsafe_allow_html=True)
        st.markdown(f"### Average Confidence: <span style='color:white'>{confidence:.2f}%</span>", unsafe_allow_html=True)
        st.write("**Average class probabilities from all models:**")
        for label, prob in zip(CLASS_LABELS, avg_probs):
            st.write(f"- **{label}**: {prob*100:.2f}%")
else:
    st.info("Please upload an eye image to begin analysis")


Writing app.py


In [4]:
from pyngrok import ngrok
import subprocess
import time

ngrok.set_auth_token("358HfFSjXSmlojBiorl4mNse6xs_MJS8wgAkf3qAxFhZUVnY")
!pkill -f streamlit
subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"])
time.sleep(7)
public_url = ngrok.connect(8501)
print("\n" + "="*60)
print(f"🎉 Your Cataract Detection site is LIVE!\n🌐 Public URL: {public_url}\n")
print("Keep this Colab tab open while using your public site.")
print("="*60 + "\n")



🎉 Your Cataract Detection site is LIVE!
🌐 Public URL: NgrokTunnel: "https://von-operatable-verbatim.ngrok-free.dev" -> "http://localhost:8501"

Keep this Colab tab open while using your public site.

